In [37]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from datetime import datetime
import ta
# import tensorflow.compat.v1.keras.layers.CuDNNLSTM as CuDNNLSTM

In [38]:
import tensorflow as tf

# Set TensorFlow to only allocate memory on the GPU when needed
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the first GPU
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPU


In [39]:
# Load the Bitcoin price data
bitcoin_data = pd.read_csv(r'C:\Users\MrBios\Documents\Development\test\csv\bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')

# Drop missing values
bitcoin_data.dropna(inplace=True)


In [40]:
# Keep the necessary columns
bitcoin_data = bitcoin_data[['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume_(BTC)', 'Volume_(Currency)']]
bitcoin_data = bitcoin_data.tail(10000)

In [41]:
# Convert Timestamp to datetime format
bitcoin_data['Timestamp'] = pd.to_datetime(bitcoin_data['Timestamp'], unit='s')

# Compute RSI
bitcoin_data['RSI'] = ta.momentum.rsi(close=bitcoin_data["Close"], fillna=True)

# Compute MACD
bitcoin_data['MACD'] = ta.trend.macd(close=bitcoin_data["Close"], fillna=True)
bitcoin_data['MACD_signal'] = ta.trend.macd_signal(close=bitcoin_data["Close"], fillna=True)
bitcoin_data['MACD_diff'] = ta.trend.macd_diff(close=bitcoin_data["Close"], fillna=True)


In [42]:
# Prepare the data for training
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(bitcoin_data.drop(columns=['Timestamp']))


In [43]:
time_step = 60
num_features = scaled_data.shape[1]

In [44]:
X = []
y = []

# Create sequences of data with a given time step
for i in range(time_step, len(scaled_data) - 30):
    X.append(scaled_data[i - time_step:i, :])
    y.append(scaled_data[i:i + 30, :])

X = np.array(X)
y = np.array(y)

In [45]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [54]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], num_features)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=y_train.shape[1]))

In [55]:
# Define custom loss function for multiple features
def multi_feature_mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

In [56]:
# Compile the model with custom loss function
model.compile(optimizer='adam', loss=multi_feature_mse)


In [57]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/50


ValueError: in user code:

    File "c:\Users\MrBios\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\MrBios\AppData\Local\Temp\ipykernel_23632\2879125739.py", line 2, in multi_feature_mse  *
        return tf.reduce_mean(tf.square(y_true - y_pred), axis=[1, 2])

    ValueError: Dimensions must be equal, but are 10 and 30 for '{{node multi_feature_mse/sub}} = Sub[T=DT_FLOAT](IteratorGetNext:1, sequential_4/dense_4/BiasAdd)' with input shapes: [?,30,10], [?,30].


In [ ]:
# Save the model and weights
version = "V6"
now = datetime.now()
format_date = now.strftime("%Y-%m-%d-%H-%M")
model.save(f'{version}/models/2-{version}-model_{format_date}.h5')
model.save_weights(f'{version}/models/2-{version}-weights_{format_date}.h5')


In [ ]:
# Make predictions
predictions = model.predict(X_test)

# Scale the predictions back to original values
predictions = scaler.inverse_transform(predictions)

In [ ]:
# Predict the RSI and MACD
rsi_predictions = predictions[:, 5]
macd_predictions = predictions[:, 6]

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
# Plot the actual and predicted values
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
ax1.plot(y_test[0][:, 3], label='Actual')
ax1.set_title('Actual Bitcoin Price')
ax1.set_xlabel('Time')
ax1.set_ylabel('Price')
ax1.legend()

ax2.plot(predictions[0][:, 3], label='Predicted')
ax2.set_title('Predicted Bitcoin Price')
ax2.set_xlabel('Time')
ax2.set_ylabel('Price')
ax2.legend()

plt.tight_layout()
plt.show()